In [10]:
## This module build the model on ID3 ML algorithm and test on 20% datasets.

##  "PortScan.csv" file is required for the operation of the program.
##  "PortScan.csv" file must be located in the same directory as the program.

from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import joblib

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import os
import pandas as pd
import csv
import time
import warnings
import math
warnings.filterwarnings("ignore")

result="./results/results_2.csv" #a CSV file is named in which the results are saved.
csv_files=os.listdir("attacks/Train")# CSV files names: #The names of the files in the attacks folder are taken and assigned to a list (csv_files).
# csv_files=list.append("portScan_train.csv")
print("Files which are used to train the model:",csv_files)
path="./attacks/Train/"
repetition=10  # 10 times each ML algo apply on each file.


def folder(f_name): #this function creates a folder named "results" and "result_graph_1" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

folder_name="./results/"
folder(folder_name)
folder_name="./results/result_graph_2/"
folder(folder_name)


#The machine learning algorithms to be used are defined in a dictionary (ml_list).
ml_list={
"ID3" :DecisionTreeClassifier(max_depth=5,criterion="entropy")
}


clf = ml_list["ID3"]

features={"portScan_train":["Flow Bytes/s","Total Length of Fwd Packets","Fwd IAT Total","Flow Duration","Label"],
         }
seconds=time.time()#time stamp for all processing time



with open(result, "w", newline="",encoding="utf-8") as f:#a CSV file is created to save the results obtained.
    wrt = csv.writer(f)
    wrt.writerow(["File","ML algorithm","accuracy","Precision", "Recall" , "F1-score","Time"])




for j in csv_files: #this loop runs on the list containing the filenames.Operations are repeated for all attack files
    a=[]
    
    feature_list=list(features[j[0:-4]])
    print("Features are used to train and test models:")
    print(feature_list,"\n\n")
    
    print ('%-17s %-17s  %-15s %-15s %-15s %-15s %-15s' % ("File","ML algorithm","accuracy","Precision", "Recall" , "F1-score","Time"))# print output header
    df=pd.read_csv(path+j,usecols=feature_list)#read an attack file.
    df=df.fillna(0)
    print("Size of dataset=",df.size)
    attack_or_not=[]
    for i in df["Label"]: #it changes the normal label to "1" and the attack tag to "0" for use in the machine learning algorithm
        
        if i =="BENIGN":
            attack_or_not.append(1)
        else:
            attack_or_not.append(0)     
           # attack_or_not.append(1) 
    df["Label"]=attack_or_not

    y = df["Label"] #this section separates the label and the data into two separate pieces, as Label=y Data=X 
    del df["Label"]
    feature_list.remove('Label')
    X = df[feature_list]

    
    for ii in ml_list: #this loop runs on the list containing the machine learning algorithm names. Operations are repeated for all the 7 algorithm
        precision=[]
        recall=[]
        f1=[]
        accuracy=[]
        t_time=[]
        overall_predict=[] # make it gobally
        for i in range(repetition): # This loop allows cross-validation and machine learning algorithm to be repeated 10 times
            second=time.time()#time stamp for processing time

            # cross-validation
            X_train, X_test, y_train, y_test = train_test_split(X, y,#  data (X) and labels (y) are divided into 2 parts to be sent to the machine learning algorithm (80% train,%20 test). 
                test_size = 0.20, random_state = repetition)#  So, in total there are 4 tracks: training data(X_train), training tag (y_train), test data(X_test) and test tag(y_test).


            #machine learning algorithm is applied in this section
        #    clf = ml_list[ii]           #choose algorithm from ml_list dictionary                                                                          
            clf.fit(X_train, y_train)     
            predict =clf.predict(X_test) 
            
            overall_predict.append(np.mean(predict))
            #makes "classification report" and assigns the precision, f-measure, and recall values.s.    
                  
            f_1=f1_score(y_test, predict, average='macro')
            pr=precision_score(y_test, predict, average='macro')
            rc=recall_score(y_test, predict, average='macro')

            
            # Experimenting
#             if(np.mean(predict)<0.5):
#                 print("Malicious Network, Disconnect from the network")
            
            
            
            precision.append(float(pr))
            recall.append(float(rc))
            f1.append(float(f_1))
            accuracy.append(clf.score(X_test, y_test))
            t_time.append(float((time.time()-second)) )


        print ('%-17s %-17s  %-15s %-15s %-15s %-15s %-15s' % (j[0:-4],ii,str(round(np.mean(accuracy),2)),str(round(np.mean(precision),2)), 
            str(round(np.mean(recall),2)),str(round(np.mean(f1),2)),str(round(np.mean(t_time),4))))#the result of the ten repetitions is printed on the screen.

        with open(result, "a", newline="",encoding="utf-8") as f: # all the values found are saved in the opened file.
            wrt = csv.writer(f)
            for i in range(0,len(t_time)):
                wrt.writerow([j[0:-4],ii,accuracy[i],precision[i],recall[i],f1[i],t_time[i]])#file name, algorithm name, precision, recall and f-measure are writed in CSV file
        a.append(f1)
        
        
        # Generating network health report
#----------------------------------------------------------------------------------------------------
        print("\n\n")
        print(110*"*","\nReport\n")
        if(np.mean(overall_predict)<0.5):
            print("Malicious Network found, Disconnect quickly")
        else:
            print("Safe Network")


    print("\n------------------------------------------------------------------------------------------------------\n\n")

joblib.dump(clf, 'portscan_model.pkl')
print("mission accomplished!")
print("Total operation time: = ",time.time()- seconds ,"seconds")

Files which are used to train the model: ['portScan_train.csv']
Features are used to train and test models:
['Flow Bytes/s', 'Total Length of Fwd Packets', 'Fwd IAT Total', 'Flow Duration', 'Label'] 


File              ML algorithm       accuracy        Precision       Recall          F1-score        Time           
Size of dataset= 1432015
portScan_train    ID3                1.0             1.0             1.0             1.0             0.3378         



************************************************************************************************************** 
Report

Malicious Network found, Disconnect quickly

------------------------------------------------------------------------------------------------------


mission accomplished!
Total operation time: =  4.068782091140747 seconds


In [ ]:
# This module is to extract 4 relevant features from dataset .
import pandas as pd
import random
# read the original CSV file
df = pd.read_csv("attacks/Test/benign_4.csv")


df["Flow Bytes/s"]=df["Flow Byts/s"]
df["Total Length of Fwd Packets"]=df["TotLen Fwd Pkts"]
df["Fwd IAT Total"]=df["Fwd IAT Tot"]

# select the desired columns
selected_columns = ["Flow Bytes/s","Total Length of Fwd Packets","Fwd IAT Total","Flow Duration"]
new_df = df[selected_columns]
print(new_df)
# save the new dataframe to a CSV file
#random_number = random.randint(1,100)
new_df.to_csv(f"./attacks/Test/benign_4.csv", index=False)


In [ ]:
## This module only for testing purpose on 4 features on above built model clf

##  "PortScan.csv" file is required for the operation of the program.
##  "PortScan.csv" file must be located in the same directory as the program.

##  As the program display output data include: file name, machine learning algorithm name, accuracy,Precision, Recall, F1-score,Time
##  the program will create a CSV file that prints the results and a folder containing graphics.

##  the some codes parts used for calculation and graphing are taken from the following site.
##  http://scikit-learn.org


from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score


import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import os
import pandas as pd
import csv
import time
import warnings
import math
warnings.filterwarnings("ignore")

result="./results/results_2.csv" #a CSV file is named in which the results are saved.
csv_files=os.listdir("attacks/Test")# CSV files names: #The names of the files in the attacks folder are taken and assigned to a list (csv_files).
print("Files which are used to test the model:", csv_files)
path="./attacks/Test/"
repetition=1  # 10 times each ML algo apply on each file.


def folder(f_name): #this function creates a folder named "results" and "result_graph_1" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

folder_name="./results/"
folder(folder_name)
folder_name="./results/result_graph_2/"
folder(folder_name)


#The machine learning algorithms to be used are defined in a dictionary (ml_list).
ml_list={
"ID3" :DecisionTreeClassifier(max_depth=5,criterion="entropy")
}



seconds=time.time()#time stamp for all processing time



for j in csv_files: #this loop runs on the list containing the filenames.Operations are repeated for all attack/Test files
    a=[]
#     print(j)
#     feature_list=list(features[j[0:-4]])
    feature_list=["Flow Bytes/s","Total Length of Fwd Packets","Fwd IAT Total","Flow Duration"]
         
    print(f"Features are used to test models: {j}")
    print(feature_list,"\n\n")
    
#     print ('%-17s %-17s  %-15s %-15s %-15s %-15s %-15s' % ("File","ML algorithm","accuracy","Precision", "Recall" , "F1-score","Time"))# print output header
    df=pd.read_csv(path+j,usecols=feature_list)#read an attack file.
    df=df.fillna(0)
    print("Size of dataset=",df.size)
    
#    print(df)
    
    X = df[feature_list]

    t_time=[]
    
    second=time.time()#time stamp for processing time
 
    predict =clf.predict(X) 

    t_time.append(float((time.time()-second)) )


        # Generating network health report
#----------------------------------------------------------------------------------------------------
    print("\n\n")
#     print(110*"*","\nReport For {j} File\n")
    print(110*"*")
    print(f"\nReport For {j} File\n")
#         if(np.mean(overall_predict)<0.5):
    print(predict)
    if(np.mean(predict)<0.5):
        print("Malicious Network found, Disconnect quickly")
    else:
        print("Safe Network")

 
    print("\n------------------------------------------------------------------------------------------------------\n\n")
    
print("mission accomplished!")
print("Total operation time: = ",time.time()- seconds ,"seconds")